# Projeto 2 - Ciência dos Dados

Nome: _____

Nome: _____

Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [9]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [10]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

* Conta: ***[Preencha aqui o id da sua conta. Ex: @fulano ]***

In [11]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano
#Conta: @gabrielpfm

#leitura do arquivo no formato JSON
#with open('auth.pass') as fp:    
 #   data = json.load(fp)

#Configurando a biblioteca. Não modificar
#auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
#auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [12]:
#Produto escolhido:
#produto = 'Nintendo'

#Quantidade mínima de mensagens capturadas:
#n = 750
#Quantidade mínima de mensagens para a base de treinamento:
t = 450

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [13]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
#i = 1
#msgs = []
#for msg in tweepy.Cursor(api.search, q=produto, lang=lang,tweet_mode="extended").items():    
    #msgs.append(msg.full_text.lower())
 #   i += 1
 # #  if i > n:
   #     break

#Embaralhando as mensagens para reduzir um possível viés
#shuffle(msgs)

NameError: name 'auth' is not defined

Salvando os dados em uma planilha Excel:

In [ ]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
#if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
   # writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
  #  dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
   # dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

  #  dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
  #  dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
  #  writer.save()

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [18]:
import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    import string
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed

In [19]:
Excel = pd.read_excel("Nintendo.xlsx", sheet_name="Treinamento")


In [20]:
#Transformando Excel em 3 textos divididos por categoria
trein_cat0 = " ".join(Excel[Excel["Classificação"]==0].Treinamento)
trein_cat1 = " ".join(Excel[Excel["Classificação"]==1].Treinamento)
trein_cat2 = " ".join(Excel[Excel["Classificação"]==2].Treinamento)

In [21]:
#Três'textões' após a limpeza
cat0_clean = cleanup(trein_cat0.lower())
serie0 = pd.Series(cat0_clean.split())
cat1_clean = cleanup(trein_cat1.lower())
serie1 = pd.Series(cat1_clean.split())
cat2_clean = cleanup(trein_cat2.lower())
serie2 = pd.Series(cat2_clean.split())

In [22]:
freq_abs0 = serie0.value_counts()
freq_abs1 = serie1.value_counts()
freq_abs2 = serie2.value_counts()
freq_rel0 = serie0.value_counts(True)
freq_rel1 = serie1.value_counts(True)
freq_rel2 = serie2.value_counts(True)
freq_rel = [freq_rel0,freq_rel1,freq_rel2]

In [23]:
tudo = cat0_clean+cat1_clean+cat2_clean
serie_tudo = pd.Series(tudo.split())
freq_abs_tudo = serie_tudo.value_counts()
freq_rel_tudo = serie_tudo.value_counts(True)


In [24]:
Prob_0 = freq_abs0.sum()/freq_abs_tudo.sum()
Prob_1 = freq_abs1.sum()/freq_abs_tudo.sum()
Prob_2 = freq_abs2.sum()/freq_abs_tudo.sum()
Probs = [Prob_0,Prob_1,Prob_2]

In [25]:
set0 = set(freq_rel0.index)
set1 = set(freq_rel1.index)
set2 = set(freq_rel2.index)
inter01 = set0.intersection(set1)
inter02 = set0.intersection(set2)
inter12 = set1.intersection(set2)
inter012 = set0.intersection(inter12)


In [26]:
def Classificador(tweet,freq_rel):
    categoria = 0
    maior_prob = 48765665847676
    Prob_Tweet_Categoria = {}
    Prob_Categoria_Tweet = {}
    for n in range(0,3):
        probtc = freq_rel[n][tweet.values]
        Prob_Tweet_Categoria[n] = probtc.prod()
    for n in range(0,3):
        prtoct = Prob_Tweet_Categoria[n]*Probs[n]
        Prob_Categoria_Tweet[n] = prtoct
    for categorias, probabilidade in Prob_Categoria_Tweet.items():
        if probabilidade < maior_prob:
            maior_prob = probabilidade
            
            categoria = categorias
    return categoria
        


In [27]:
#Entrada do Tweet
def Limpa_Tweet(Excel_Lido,Coluna,freq_rel):
    indice = 0
    Excel_Lido.loc[:,"Classificação"] = 0
    for tweets in Excel_Lido.Teste:
        tweet_limpo = cleanup(tweets.lower())
        tweet = pd.Series(tweet_limpo.split())
        classificado = Classificador(tweet,freq_rel)
        Excel_Lido.loc[indice,"Classificação"] = classificado
        indice +=1
    return Excel_Lido
    

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

In [55]:
Excel_Teste = pd.read_excel("Nintendo.xlsx", sheet_name="Teste")

manual =[]
for classficacao in Excel_Teste["Classificação"]:
    manual.append(classficacao)


teste = "Teste"
Excel_Classificado = Limpa_Tweet(Excel_Teste,teste,freq_rel)
Excel_Classificado;


In [56]:
classificador = []
for classficacao in Excel_Classificado["Classificação"]:
    classificador.append(classficacao)



In [57]:
Teste_de_Performance = {"Certo":0,"Errado":0}
for i in range(len(manual)):
    if classificador[i] == manual[i]:
        Teste_de_Performance["Certo"]+=1
    else:
        Teste_de_Performance["Errado"]+=1
print(Teste_de_Performance)

{'Certo': 201, 'Errado': 199}


___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**